In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential, layers
from keras.layers import Embedding,LSTM, Dense,GlobalAveragePooling2D, Dropout


df = pd.read_csv('reddit_depression_dataset.csv')
print(df.size)
print("Пропуски у даних:\n", df.isnull().sum())
df = df.dropna()


columns = ['subreddit','body']

df_c = pd.DataFrame( columns=columns)

stop_words = stopwords.words("english")

def clean_text(text):
    text = re.sub("\d", " ", text)
    text = text.lower()
    text = re.sub("https*\S+", " ", text)
    text = re.sub("@\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

df_c['body'] = df['body'].apply(clean_text) 
df_c['subreddit'] = df['subreddit']
df_c1 = df_c.copy()
print(df_c1)

sr = ['DeepThoughts','happy','SuicideWatch']
df_c = df_c[df_c['subreddit'].isin(sr)]

# Виводимо результат
print(df_c)

/var/folders/bc/rcgkn6p91vq7pnqphwwsg_380000gn/T/ipykernel_43246/4119247623.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('reddit_depression_dataset.csv')


19766224
Пропуски у даних:
 Unnamed: 0           3
subreddit           20
title               23
body            461051
upvotes             64
created_utc        106
num_comments    113977
label              106
dtype: int64
            subreddit                                               body
1        DeepThoughts  anyway human morality joke long abscence hope ...
2        DeepThoughts  hello new guy charge besides thegood ofc bring...
4        DeepThoughts  could take one cell body kill would still yet ...
5        DeepThoughts             personally think infinite always grow 
7        DeepThoughts  hello christmas hosting contest ever post best...
...               ...                                                ...
2470773    depression  argued mom hour ago always feels attacked ever...
2470774    depression  ok basically non friend family invited christm...
2470775    depression                         today gonna ok push little
2470776    depression  know recently realized

In [44]:
texts = df_c['body'].values  
labels = df_c['subreddit'].values  

tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

maxlen = 200  
X = pad_sequences(sequences, maxlen=maxlen)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y)  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = keras.Sequential()

model.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
model.add(LSTM(128,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1],activation = 'softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




/Users/akh/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [45]:
# Навчання моделі
history = model.fit(X_train, y_train, 
                    epochs=5, 
                    batch_size=128, 
                    validation_split=0.2,
                    shuffle = True,
                    verbose=1)
model.summary()

Epoch 1/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 272s 288ms/step - accuracy: 0.9114 - loss: 0.3149 - val_accuracy: 0.9523 - val_loss: 0.1512
Epoch 2/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 271s 288ms/step - accuracy: 0.9624 - loss: 0.1209 - val_accuracy: 0.9551 - val_loss: 0.1381
Epoch 3/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 276s 294ms/step - accuracy: 0.9708 - loss: 0.0951 - val_accuracy: 0.9539 - val_loss: 0.1534
Epoch 4/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 276s 293ms/step - accuracy: 0.9761 - loss: 0.0781 - val_accuracy: 0.9536 - val_loss: 0.1523
Epoch 5/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 272s 290ms/step - accuracy: 0.9802 - loss: 0.0656 - val_accuracy: 0.9504 - val_loss: 0.1808


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,260,107 (16.25 MB)

 Trainable params: 1,420,035 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,840,072 (10.83 MB)

In [46]:
score = model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])


1175/1175 ━━━━━━━━━━━━━━━━━━━━ 50s 42ms/step - accuracy: 0.9526 - loss: 0.1673
Test loss: 0.1706639528274536
Test accuracy: 0.9516218900680542


Pretrained word embeddings

In [51]:
embeddings_index = {}
with open("glove.6B.200d.txt", encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 200
word_index = tokenizer.word_index
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector



In [52]:
model = Sequential()
model.add(Embedding(input_dim=10000, 
                    output_dim=embedding_dim, 
                    input_length=maxlen, 
                    weights=[embedding_matrix], 
                    trainable=False))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [53]:
history = model.fit(X_train, y_train, 
                    epochs=5, 
                    batch_size=128, 
                    validation_split=0.2,
                    shuffle=True,
                    verbose=1)

Epoch 1/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 255s 270ms/step - accuracy: 0.9193 - loss: 0.2763 - val_accuracy: 0.9366 - val_loss: 0.1885
Epoch 2/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 247s 263ms/step - accuracy: 0.9477 - loss: 0.1637 - val_accuracy: 0.9506 - val_loss: 0.1489
Epoch 3/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 240s 255ms/step - accuracy: 0.9541 - loss: 0.1402 - val_accuracy: 0.9535 - val_loss: 0.1424
Epoch 4/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 251s 267ms/step - accuracy: 0.9569 - loss: 0.1312 - val_accuracy: 0.9542 - val_loss: 0.1367
Epoch 5/5
940/940 ━━━━━━━━━━━━━━━━━━━━ 248s 264ms/step - accuracy: 0.9603 - loss: 0.1195 - val_accuracy: 0.9540 - val_loss: 0.1384


In [54]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1175/1175 ━━━━━━━━━━━━━━━━━━━━ 52s 44ms/step - accuracy: 0.9577 - loss: 0.1272
Test loss: 0.13214819133281708
Test accuracy: 0.9560392498970032
